# Facial Emotion Recognition Project

### Imports/Setup

In [ ]:
# Imports
from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf

# Set seed
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

### Load data

In [ ]:
# Training/validation/testing data
ds_train_ = image_dataset_from_directory(
    'images/train',
    labels='inferred',
    label_mode='categorical',
    image_size=[48, 48],
    color_mode='grayscale',
    seed=31415,
    validation_split=0.3,
    subset='training',
    shuffle=True
)
ds_valid_ = image_dataset_from_directory(
    'images/train',
    labels='inferred',
    label_mode='categorical',
    image_size=[48, 48],
    color_mode='grayscale',
    seed=31415,
    validation_split=0.3,
    subset='validation',
    shuffle=False
)
ds_test_ = image_dataset_from_directory(
    'images/test',
    labels='inferred',
    label_mode='categorical',
    image_size=[48, 48],
    interpolation='nearest',
    shuffle=True
)

In [ ]:
# Data Pipeline
def preprocess_image(image, label):
    image = tf.image.resize(image, (48, 48))
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(preprocess_image)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(preprocess_image)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_test = (
    ds_test_
    .map(preprocess_image)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [ ]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
        'images/test',
        target_size=(48, 48),
        batch_size=32,
        class_mode='categorical')

### Data Augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
])

### Model

In [ ]:
# Sequential modeel
model = Sequential()

# Data augmentation
model.add(data_augmentation)

# Block 1
model.add(Conv2D(input_shape=(48,48,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.1))
# Block 2
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.1))
# Block 3
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.1))
# Block 4
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# Classification head
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=7,activation="softmax"))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=0.01)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

### Callbacks (early stopping + model checkpoint saving)

In [ ]:
# early stopping
es = EarlyStopping(monitor='val_accuracy', min_delta= 0.001 , patience= 5, verbose= 1, mode='auto')
# model check point
mc = ModelCheckpoint(filepath="models/model.h5", monitor= 'val_accuracy', verbose= 1, save_best_only= True, mode = 'auto')
# puting call back in a list 
call_back = [es, mc]

### Training model + Plot learning curves

In [ ]:
history = model.fit(
    ds_train_,
    validation_data=ds_valid_,
    epochs=25,
    callbacks=[es,mc]
)

# Plot learning curves
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

### Metrics

In [ ]:
predictions = model.predict_generator(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

cm=confusion_matrix(true_classes, yPredictions)
print(cm)

In [ ]:
cm_df = pd.DataFrame(cm, range(7))
sns.heatmap(cm_df, annot=True)
plt.show()

### Saving model

In [ ]:
model.save('models/model.h5')
# Saving as json file + saving weights
model_json = model.to_json()  
with open("model.json", "w") as json_file:  
    json_file.write(model_json)  
model.save_weights("model_weights.h5")

In [ ]:
from keras.models import load_model
model = load_model("model2.h5")